In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/MyDrive/새싹프로젝트_텍스트

/content/drive/MyDrive/새싹프로젝트_텍스트


In [4]:
ls

'circulus koelectra-polite-v1.ipynb'   Mecab-ko-for-Google-Colab/
 concat.csv                            model_trained.pt
 crwaling.ipynb                        modified_mecab_add_kobert_with_wandb_sweep.ipynb
 data_processed.csv                   'monologg kobert.ipynb'
 df_pre.csv                            pt_dataset/
 han_article.csv                       Untitled
 han_rename.csv                        wandb/
 huggingface.ipynb                     wandb_use_news.ipynb
 KoELECTRA.ipynb                       셀레니움을_쓰지않았을때.html
 mecab_add_kobert.ipynb                한겨레전처리.ipynb


In [ ]:
!pip install transformers

In [12]:
!pip install transformers torch pandas tqdm



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.7 MB/s eta 0:00:00


# 데이터 불러오기 ( 지금은 han 데이터만 씀 -> 나중에 전체꺼 오면 바꿔서 실행.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

from transformers import ElectraForSequenceClassification, ElectraTokenizer

# 사전 훈련된 모델 로드
model = ElectraForSequenceClassification.from_pretrained("copycats/koelectra-base-v3-generalized-sentiment-analysis")

# classifier의 out_proj 레이어의 출력 차원을 4(레이블 수)로 조정
import torch.nn as nn

model.classifier.out_proj = nn.Linear(in_features=model.classifier.dense.out_features, out_features=4)

# 모델의 num_labels 속성 업데이트
model.num_labels = 4


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
# CSV 파일 로드
df = pd.read_csv("han_rename.csv", encoding="utf-8")
# NaN 값이나 None을 포함하는 행 제거
df.dropna(subset=['text', 'label'], inplace=True)

# 레이블 인코딩
label_dict = {'정치': 0, '사회': 1, '경제': 2, '국제': 3}
df['label'] = df['label'].map(label_dict)

# 훈련 데이터와 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(df['text'], df['label'], test_size=0.1, random_state=42)


# 한글자 처리 코드

In [ ]:
#페기
def remove_single_characters(text):
    # 공백을 기준으로 텍스트를 단어로 분리
    words = text.split()
    # 1글자가 아닌 단어만 선택
    filtered_words = [word for word in words if len(word) > 1]
    # 다시 텍스트로 결합
    return ' '.join(filtered_words)


In [ ]:
from transformers import AutoTokenizer

# 사전 훈련된 모델에 맞는 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("copycats/koelectra-base-v3-generalized-sentiment-analysis")


# 훈련 데이터와 검증 데이터에 함수 적용
X_train_filtered = X_train.apply(remove_single_characters)
X_val_filtered = X_val.apply(remove_single_characters)

# 필터링된 데이터에 대해 토크나이징 수행 (attention_mask 포함)
train_encodings = tokenizer(X_train_filtered.tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')
val_encodings = tokenizer(X_val_filtered.tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

NameError: name 'X_train' is not defined

In [ ]:

# TensorDataset 생성
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(y_train.values))
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(y_val.values))

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)


In [ ]:
!pip install pandas tqdm


# 클래스 화


In [5]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from transformers import ElectraForSequenceClassification, ElectraTokenizer
from torch.optim import AdamW
from tqdm.auto import tqdm
import os

import torch.nn as nn

#실패한1 글자 지우기 코드 버림.
def remove_single_characters(text):
    # 공백을 기준으로 텍스트를 단어로 분리
    words = text.split()
    # 1글자가 아닌 단어만 선택
    filtered_words = [word for word in words if len(word) > 1]
    # 다시 텍스트로 결합
    return ' '.join(filtered_words)

class DataPreprocessor:
    def __init__(self, tokenizer,label_dict:dict = {'정치': 0, '사회': 1, '경제': 2, '국제': 3}):
        self.tokenizer = tokenizer
        self.label_dict = label_dict

    def encode_labels(self, df):
        df['label'] = df['label'].map(self.label_dict)
        return df

    def split_data(self, df):
        return train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

    def preprocess_and_tokenize(self, texts):
        #texts_filtered = texts.apply(remove_single_characters)  # 이전에 정의한 함수 사용
        encodings = self.tokenizer(texts.tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')
        return encodings

    def create_data_loader(self, df):
        df = self.encode_labels(df)
        X_train, X_val, y_train, y_val = self.split_data(df)

        train_encodings = self.preprocess_and_tokenize(X_train)
        val_encodings = self.preprocess_and_tokenize(X_val)

        train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(y_train.values))
        val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(y_val.values))

        train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=8)

        return train_loader, val_loader


class ModelTrainer:
    def __init__(self, model, device):
        self.model = model
        self.device = device
        self.model_save_index = 0
    def train(self, train_loader, optimizer):
        self.model.train()
        for batch in tqdm(train_loader):
            batch = tuple(t.to(self.device) for t in batch)
            inputs = {
                'input_ids': batch[0],
                'attention_mask': batch[1],
                'labels': batch[2]
            }
            optimizer.zero_grad()
            outputs = self.model(**inputs)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

    def evaluate(self, val_loader):
        self.model.eval()
        predictions, true_vals = [], []
        for batch in val_loader:
            batch = tuple(t.to(self.device) for t in batch)
            inputs = {
                'input_ids': batch[0],
                'attention_mask': batch[1]
            }
            with torch.no_grad():
                outputs = self.model(**inputs)

            logits = outputs.logits
            logits = logits.detach().cpu().numpy()
            label_ids = batch[2].cpu().numpy()
            predictions.append(np.argmax(logits, axis=1).flatten())
            true_vals.append(label_ids.flatten())

        predictions = np.concatenate(predictions)
        true_vals = np.concatenate(true_vals)
        return accuracy_score(true_vals, predictions)

    def run_training_loop(self, train_loader, val_loader, epochs=7):
        optimizer = AdamW(self.model.parameters(), lr=5e-5)

        for epoch in range(epochs):
          print(f"Epoch {epoch+1}/{epochs}")
          self.train(train_loader, optimizer)
          val_acc = self.evaluate(val_loader)
          print(f"Validation Accuracy: {val_acc}")
          filename = f"FacebookAI_roberta{self.model_save_index}.pt"
          self.save_model(os.path.join('pt_dataset', filename))
          self.model_save_index += 1  # 파일 이름이 중복되지 않도록 인덱스 증가
    def save_model(self, file_path):
      # 모델 상태 저장
      torch.save(self.model.state_dict(), file_path)
      print(f'Model saved to {file_path}')




In [6]:
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer
import torch

# 모델과 토크나이저 초기화
tokenizer = XLMRobertaTokenizer.from_pretrained('FacebookAI/xlm-roberta-base')
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=4)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [ ]:

label_dict = {'정치': 0, '사회': 1, '경제': 2, '국제': 3}


# CSV 파일 로드
df = pd.read_csv("data_processed.csv", encoding="utf-8")
# NaN 값이나 None을 포함하는 행 제거
df.dropna(subset=['text', 'label'], inplace=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
preprocessor = DataPreprocessor(tokenizer,label_dict)
train_loader, val_loader = preprocessor.create_data_loader(df)
trainer = ModelTrainer(model, device)
trainer.run_training_loop(train_loader, val_loader,epochs=15)


Epoch 1/15


  0%|          | 0/192 [00:00<?, ?it/s]